## **Libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/cwitkowitz/guitar-transcription-continuous.git
%cd guitar-transcription-continuous
!pip install -r requirements.txt
!pip install -e .
!pip install resampy

Cloning into 'guitar-transcription-continuous'...
remote: Enumerating objects: 579, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 579 (delta 25), reused 21 (delta 21), pack-reused 552 (from 1)
Receiving objects: 100% (579/579), 157.99 KiB | 1.11 MiB/s, done.
Resolving deltas: 100% (379/379), done.
/content/guitar-transcription-continuous
  Cloning https://github.com/cwitkowitz/guitar-transcription-with-inhibition (to revision main) to /tmp/pip-req-build-12gygufj
  Running command git clone --filter=blob:none --quiet https://github.com/cwitkowitz/guitar-transcription-with-inhibition /tmp/pip-req-build-12gygufj
  Resolved https://github.com/cwitkowitz/guitar-transcription-with-inhibition to commit e611c1dc9b7340d35c9a697d1658b3b2afb3978a
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

Obtaining file:///content/guitar-transcription-continuous
  Preparing metadata (setup.py) ... done
  Running setup.py develop for guitar-transcription-continuous
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.6 MB/s eta 0:00:00


In [3]:
!pip uninstall -y numpy
!pip install numpy==1.24.4

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 40.3 MB/s eta 0:00:00
^C


In [4]:
import os
import torch
import librosa
import numpy as np
import csv

from guitar_transcription_continuous.models.fretnet import FretNet
from amt_tools.features import HCQT
from amt_tools.transcribe import ComboEstimator, TablatureWrapper, StackedOffsetsWrapper, StackedNoteTranscriber
from guitar_transcription_continuous.estimators import StackedPitchListTablatureWrapper
import amt_tools.tools as tools
import guitar_transcription_continuous.utils as utils
import torch.serialization

/usr/local/lib/python3.11/dist-packages/amt_tools/features/stream.py:16: RuntimeWarning: Could not import keyboard, likely because an X connection could not be acquired.
  warnings.warn('Could not import keyboard, likely because an X ' +
/usr/local/lib/python3.11/dist-packages/amt_tools/features/stream.py:22: RuntimeWarning: Could not import sounddevice. Please install PortAudio and try again.
  >>> sudo apt-get install libportaudio2
  warnings.warn('Could not import sounddevice. Please install PortAudio and ' +


In [5]:
from amt_tools.inference import run_offline

In [6]:
model_path = "/content/drive/MyDrive/Automatic Guitar Transcription/nootebooks/feature_extraction/model-2000.pt"
audio_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_mic"
jams_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/annotation"
output_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/feature_extraction/fret_string_inference_fretnet"
os.makedirs(output_dir, exist_ok=True)

torch.serialization.add_safe_globals({'FretNet': FretNet})

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(model_path, map_location=device, weights_only=False)
model.eval()
model.change_device(0)

## **Pred Single Audio**

In [7]:
def setup_estimator(model):
    return ComboEstimator([
        TablatureWrapper(profile=model.profile),
        StackedOffsetsWrapper(profile=model.profile),
        StackedNoteTranscriber(profile=model.profile),
        StackedPitchListTablatureWrapper(
            profile=model.profile,
            multi_pitch_key=tools.KEY_TABLATURE,
            multi_pitch_rel_key=utils.KEY_TABLATURE_REL
        )
    ])

In [8]:
def predict_single_full(audio_path, model, sample_rate=24000):
    import librosa
    from collections import defaultdict

    audio, _ = librosa.load(audio_path, sr=sample_rate)

    data_proc = HCQT(
        sample_rate=sample_rate,
        hop_length=512,
        fmin=librosa.note_to_hz('E2'),
        harmonics=[0.5, 1, 2, 3, 4, 5],
        n_bins=144,
        bins_per_octave=36
    )

    feats = data_proc.process_audio(audio)
    times = data_proc.get_times(audio)

    frame_len = feats.shape[-1]
    trimmed_len = frame_len - (frame_len % 9)
    feats = feats[:, :, :trimmed_len]
    times = times[:trimmed_len]

    features = {
        tools.KEY_FEATS: feats,
        tools.KEY_TIMES: times
    }

    estimator = setup_estimator(model)
    predictions = run_offline(features, model, estimator)

    fret_string_list = []
    notes = predictions[tools.KEY_NOTES]
    string_to_midi = {
        5: 64,  # E4 (en tiz)
        4: 59,  # B3
        3: 55,  # G3
        2: 50,  # D3
        1: 45,  # A2
        0: 40   # E2 (en pes)
    }

    fret_string_list = []
    for string, (midi_list, time_list) in notes.items():
        if string not in string_to_midi:
            print(f"⚠️ Geçersiz tel (string): {string} — atlandı.")
            continue
        open_note = string_to_midi[string]
        for midi, (onset, offset) in zip(midi_list, time_list):
            fret = int(midi) - open_note
            fret_string_list.append({
                "string": string,
                "fret": fret,
                "midi": int(midi),
                "onset": onset,
                "offset": offset,
                "duration": offset - onset
            })

    return {
        "fret_string": fret_string_list,
        "notes": notes,
        "tablature": predictions.get("tablature"),
        "tablature_rel": predictions.get("tablature_rel"),
        "multi_pitch": predictions.get("multi_pitch"),
        "onsets": predictions.get("onsets"),
        "offsets": predictions.get("offsets"),
        "times": predictions.get("times"),
        "pitch_list": predictions.get("pitch_list"),
    }

In [9]:
test_file = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_pickup_mix/00_BN1-129-Eb_solo_mix.wav"

results = predict_single_full(test_file, model)

print("\n🎸 Fret-String Tahminleri:")
for row in results["fret_string"]:
    print(f"String {row['string']} | Fret {row['fret']} | MIDI {row['midi']} | Onset: {row['onset']:.2f}s → Offset: {row['offset']:.2f}s")

print("\n🧩 Tablature shape:", results["tablature"].shape if results["tablature"] is not None else "Yok")

print("🎼 Multi-pitch shape:", results["multi_pitch"].shape if results["multi_pitch"] is not None else "Yok")


🎸 Fret-String Tahminleri:
String 0 | Fret 6 | MIDI 46 | Onset: 0.00s → Offset: 0.19s
String 1 | Fret 7 | MIDI 52 | Onset: 0.47s → Offset: 0.79s
String 2 | Fret 6 | MIDI 56 | Onset: 0.00s → Offset: 0.19s
String 2 | Fret 7 | MIDI 57 | Onset: 1.90s → Offset: 2.50s
String 2 | Fret 8 | MIDI 58 | Onset: 4.20s → Offset: 4.22s
String 2 | Fret 5 | MIDI 55 | Onset: 8.81s → Offset: 8.83s
String 2 | Fret 8 | MIDI 58 | Onset: 13.50s → Offset: 13.53s
String 2 | Fret 5 | MIDI 55 | Onset: 20.54s → Offset: 20.82s
String 3 | Fret 2 | MIDI 57 | Onset: 2.77s → Offset: 2.79s
String 3 | Fret 7 | MIDI 62 | Onset: 3.22s → Offset: 3.71s
String 3 | Fret 7 | MIDI 62 | Onset: 4.44s → Offset: 4.46s
String 3 | Fret 0 | MIDI 55 | Onset: 8.81s → Offset: 9.32s
String 3 | Fret 5 | MIDI 60 | Onset: 13.03s → Offset: 13.16s
String 3 | Fret 3 | MIDI 58 | Onset: 14.87s → Offset: 14.89s
String 3 | Fret 2 | MIDI 57 | Onset: 15.38s → Offset: 15.62s
String 3 | Fret 3 | MIDI 58 | Onset: 15.57s → Offset: 15.59s
String 3 | Fret 5

In [10]:
import numpy as np

def print_ascii_tab(notes, total_duration, resolution=0.25):
    """
    :param notes: predict_single_full() fonksiyonundan gelen 'notes' dict
    :param total_duration: ses dosyasının toplam süresi (saniye)
    :param resolution: her bir sütunun temsil ettiği zaman (saniye) – örn. 0.25s
    """

    time_steps = int(total_duration / resolution)
    tab_lines = [["-" for _ in range(time_steps)] for _ in range(6)]  # 6 tel: 0 (en pes) - 5 (en tiz)

    # MIDI open string values (standard tuning EADGBE)
    string_to_midi = {
        5: 64,  # E4 (en tiz)
        4: 59,  # B3
        3: 55,  # G3
        2: 50,  # D3
        1: 45,  # A2
        0: 40   # E2 (en pes)
    }

    for string, (midi_list, time_list) in notes.items():
        if string not in string_to_midi:
            continue  # geçersiz tel
        open_note = string_to_midi[string]
        for midi, (onset, _) in zip(midi_list, time_list):
            fret = int(midi) - open_note
            if fret < 0 or fret > 30:
                continue  # olası hata

            idx = int(onset / resolution)
            if idx < time_steps:
                tab_lines[string][idx] = str(fret).rjust(2, "-") if fret < 10 else str(fret)

    # ASCII tab formatında yazdır
    print("\n🎸 Generated ASCII TAB:\n")
    string_names = ["E", "B", "G", "D", "A", "E"]
    for i in reversed(range(6)):
        line = "".join(tab_lines[i])
        print(f"{string_names[5 - i]}|{line}|")

In [11]:
test_file = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_pickup_mix/00_BN1-129-Eb_solo_mix.wav"
notes = predict_single_full(test_file, model)["notes"]

import librosa
duration = librosa.get_duration(filename=test_file)

print_ascii_tab(notes, total_duration=duration)


🎸 Generated ASCII TAB:

E|------------------------6---8---6-8---6-------------------------------------------------------|
B|---------------------------8-6------10--------6-----3-4--4--------------------------------------|
G|------------2-7-----7------------------0-----------------5-------3--2-3-5--5-5-7-5-----5---------------|
D|-6-------7---------8-------------------5-------------------8----------------------------5------|
A|--7---------------------------------------------------------------------------------------|
E|-6----------------------------------------------------------------------------------------|


/tmp/ipython-input-11-1993322072.py:5: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=test_file)


In [12]:
def compare_predictions_with_jams(jams_path, pred_notes, tolerance=0.05):
    import jams
    import numpy as np

    jam = jams.load(jams_path)
    note_annots = jam.search(namespace='note_midi')

    if len(note_annots) == 0:
        print("⚠️ Bu .jams dosyasında 'note_midi' namespace bulunamadı.")
        return

    # Tüm note_midi anotasyonlarını tek bir listede birleştir
    ref_onsets, ref_durations, ref_midis = [], [], []
    for annot in note_annots:
        for row in annot.data:
            try:
                onset = float(row.time)
                duration = float(row.duration)
                midi = int(row.value)
                ref_onsets.append(onset)
                ref_durations.append(duration)
                ref_midis.append(midi)
            except:
                continue  # hatalı veri varsa atla

    ref_onsets = np.array(ref_onsets)
    ref_midis = np.array(ref_midis)

    matches = 0
    total_preds = 0

    for string, (midi_list, time_list) in pred_notes.items():
        for midi, (onset, offset) in zip(midi_list, time_list):
            total_preds += 1
            match_found = np.any(
                (np.abs(ref_onsets - onset) <= tolerance) & (ref_midis == midi)
            )
            if match_found:
                matches += 1

    precision = matches / total_preds if total_preds else 0
    print(f"🎯 Eşleşen tahminler: {matches}/{total_preds} → Doğruluk: {precision:.2%}")

In [13]:
compare_predictions_with_jams(
    jams_path="/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/annotation/00_BN1-129-Eb_solo.jams",
    pred_notes=results["notes"]
)

🎯 Eşleşen tahminler: 34/34 → Doğruluk: 100.00%


## **Pred Single Audio**

In [14]:
import os
import json
import numpy as np

def process_and_save_all(audio_path, output_dir, model):
    def to_serializable(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (float, int, str, list, dict, bool)) or obj is None:
            return obj
        else:
            return str(obj)

    # 🔍 Tahmin al
    results = predict_single_full(audio_path, model)

    # 🔑 Eğitimde kullanılacak anahtarlar
    TRAINING_KEYS = [
        "fret_string", "tablature", "tablature_rel",
        "multi_pitch", "onsets", "offsets", "times"
    ]

    # 🔄 JSON uyumlu hale getir
    fretnet_serializable = {}
    for key in TRAINING_KEYS:
        val = results.get(key)
        if key == "fret_string" and val:
            clean_fret = []
            for r in val:
                try:
                    clean_fret.append({
                        "string": int(r["string"]),
                        "fret": int(r["fret"]),
                        "midi": int(r["midi"]),
                        "onset": float(r["onset"]),
                        "offset": float(r["offset"]),
                        "duration": float(r["duration"]),
                    })
                except:
                    continue
            fretnet_serializable[key] = clean_fret
        elif val is not None:
            try:
                fretnet_serializable[key] = to_serializable(val)
            except:
                print(f"⚠️ {key} dönüştürülemedi.")

    # 💾 JSON dosyasını kaydet
    os.makedirs(output_dir, exist_ok=True)
    filename = os.path.splitext(os.path.basename(audio_path))[0]
    fretnet_json_path = os.path.join(output_dir, f"{filename}_fretnet.json")

    with open(fretnet_json_path, "w") as f:
        json.dump(fretnet_serializable, f, indent=2)

    print(f"✅ Fretnet JSON kaydedildi: {fretnet_json_path}")

In [15]:
audio_file = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_pickup_mix/00_BN1-129-Eb_solo_mix.wav"
output_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation"

process_and_save_all(
    audio_file,
    output_dir,
    model
)

✅ Fretnet JSON kaydedildi: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/00_BN1-129-Eb_solo_mix_fretnet.json


In [16]:
import json

def check_fretnet_json(json_path):
    REQUIRED_KEYS = [
        "fret_string", "tablature", "tablature_rel",
        "multi_pitch", "onsets", "offsets", "times"
    ]

    print(f"🔍 Kontrol ediliyor: {json_path}")

    with open(json_path, "r") as f:
        data = json.load(f)

    for key in REQUIRED_KEYS:
        if key not in data:
            print(f"❌ Eksik: {key}")
        elif data[key] is None or data[key] == []:
            print(f"⚠️ Boş: {key}")
        else:
            if key == "fret_string":
                print(f"✅ {key}: {len(data[key])} adet nota (fret-string çifti)")
            elif isinstance(data[key], list):
                print(f"✅ {key}: {len(data[key])} değer")
            else:
                print(f"✅ {key}: veri mevcut")

json_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/00_BN1-129-Eb_solo_mix_fretnet.json"
check_fretnet_json(json_path)

🔍 Kontrol ediliyor: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/00_BN1-129-Eb_solo_mix_fretnet.json
✅ fret_string: 34 adet nota (fret-string çifti)
✅ tablature: 6 değer
✅ tablature_rel: 120 değer
✅ multi_pitch: 6 değer
✅ onsets: 6 değer
✅ offsets: 6 değer
✅ times: 1044 değer


In [17]:
import numpy as np
data = json.load(open(json_path))
for key in ['tablature', 'multi_pitch', 'onsets', 'offsets']:
    arr = np.array(data[key])
    print(f"{key} shape: {arr.shape}")

tablature shape: (6, 1044)
multi_pitch shape: (6, 44, 1044)
onsets shape: (6, 44, 1044)
offsets shape: (6, 44, 1044)


## **Pred For All Audios**

In [18]:
import os
from tqdm import tqdm

# 🔧 Dizinler
audio_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/audio/mono_pickup_mix"
output_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation"

# 🔍 Tüm .wav dosyalarını topla
audio_files = sorted([
    os.path.join(audio_dir, f) for f in os.listdir(audio_dir)
    if f.endswith(".wav")
])

print(f"🎧 {len(audio_files)} adet ses dosyası bulundu.\n")

# 🔁 Her ses dosyası için tahmin al ve JSON olarak kaydet
for audio_path in tqdm(audio_files, desc="🔄 Tahmin ve JSON kaydı"):
    try:
        process_and_save_all(audio_path, output_dir, model)
    except Exception as e:
        print(f"⚠️ Hata oluştu: {audio_path}\n→ {e}")

🎧 360 adet ses dosyası bulundu.



🔄 Tahmin ve JSON kaydı:   0%|          | 1/360 [00:03<18:50,  3.15s/it]

✅ Fretnet JSON kaydedildi: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/00_BN1-129-Eb_comp_mix_fretnet.json


🔄 Tahmin ve JSON kaydı:   1%|          | 2/360 [00:04<13:51,  2.32s/it]

✅ Fretnet JSON kaydedildi: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/00_BN1-129-Eb_solo_mix_fretnet.json


...

✅ Fretnet JSON kaydedildi: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/05_SS3-84-Bb_solo_mix_fretnet.json


🔄 Tahmin ve JSON kaydı: 100%|█████████▉| 359/360 [14:21<00:02,  2.95s/it]

✅ Fretnet JSON kaydedildi: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/05_SS3-98-C_comp_mix_fretnet.json


🔄 Tahmin ve JSON kaydı: 100%|██████████| 360/360 [14:24<00:00,  2.40s/it]

✅ Fretnet JSON kaydedildi: /content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/05_SS3-98-C_solo_mix_fretnet.json


In [19]:
import os

output_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation"

# .json uzantılı dosyaları filtrele
json_files = [f for f in os.listdir(output_dir) if f.endswith(".json")]

print(f"📦 Toplam {len(json_files)} adet .json dosyası bulundu.")

📦 Toplam 360 adet .json dosyası bulundu.


In [20]:
import os
import json
import numpy as np

output_dir = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation"

REQUIRED_KEYS = [
    "fret_string", "tablature", "tablature_rel",
    "multi_pitch", "onsets", "offsets", "times"
]

json_files = [f for f in os.listdir(output_dir) if f.endswith(".json")]
print(f"🔍 Toplam {len(json_files)} dosya kontrol edilecek.\n")

valid_count = 0
invalid_files = []

for fname in json_files:
    path = os.path.join(output_dir, fname)
    try:
        with open(path, "r") as f:
            data = json.load(f)

        missing_or_empty = False
        for key in REQUIRED_KEYS:
            if key not in data:
                print(f"❌ {fname}: Eksik key → {key}")
                missing_or_empty = True
            elif data[key] is None or data[key] == []:
                print(f"⚠️ {fname}: Boş key → {key}")
                missing_or_empty = True
            else:
                arr = np.array(data[key])
                if arr.size == 0:
                    print(f"⚠️ {fname}: Boş array → {key}")
                    missing_or_empty = True

        if not missing_or_empty:
            valid_count += 1
        else:
            invalid_files.append(fname)

    except Exception as e:
        print(f"💥 {fname}: Hata oluştu → {e}")
        invalid_files.append(fname)

print("\n✅ Geçerli dosya sayısı:", valid_count)
print("❌ Hatalı/Eksik dosya sayısı:", len(invalid_files))

if invalid_files:
    print("\n🚨 Hatalı dosyalar:")
    for f in invalid_files:
        print(" -", f)

🔍 Toplam 360 dosya kontrol edilecek.


✅ Geçerli dosya sayısı: 360
❌ Hatalı/Eksik dosya sayısı: 0


In [22]:
import numpy as np

suspect_files = []

for fname in json_files:
    path = os.path.join(output_dir, fname)
    try:
        with open(path, "r") as f:
            data = json.load(f)

        all_ok = True

        # 1. multi_pitch, onsets, offsets tamamen 0 mı?
        for key in ['multi_pitch', 'onsets', 'offsets']:
            arr = np.array(data[key])
            if np.all(arr == 0):
                print(f"⚠️ {fname}: {key} tamamı sıfır.")
                all_ok = False

        # 2. tablature sadece -1 veya 0 mı?
        tablature = np.array(data['tablature'])
        if np.all((tablature == -1) | (tablature == 0)):
            print(f"⚠️ {fname}: tablature boş veya kullanılmaz durumda.")
            all_ok = False

        # 3. fret_string listesi boş mu?
        if not data.get("fret_string") or len(data["fret_string"]) == 0:
            print(f"⚠️ {fname}: fret_string boş.")
            all_ok = False

        if not all_ok:
            suspect_files.append(fname)

    except Exception as e:
        print(f"💥 {fname}: JSON okunamadı → {e}")
        suspect_files.append(fname)

# Özet
print(f"\n🔍 İçerik doğrulama tamamlandı.")
print(f"✅ Anlamlı içeriğe sahip dosya sayısı: {len(json_files) - len(suspect_files)}")
print(f"❌ Anlamsız (sıfır dolu/boş) içeriğe sahip dosya sayısı: {len(suspect_files)}")

if suspect_files:
    print("\n🚨 Anlamsız veri içeren dosyalar:")
    for f in suspect_files:
        print(" -", f)


🔍 İçerik doğrulama tamamlandı.
✅ Anlamlı içeriğe sahip dosya sayısı: 360
❌ Anlamsız (sıfır dolu/boş) içeriğe sahip dosya sayısı: 0
